In [1]:
%matplotlib inline

# Transforms

データは機械学習アルゴリズムの学習に必要な最終的な処理形態であるとは限りません。  
私たちはデータを操作して学習に適した形にするために**変換**を行います。

すべてのTorchVisionデータセットには、変換ロジックを含む呼び出し可能な2つのパラメータ（`transform`は特徴の変更、`target_transform`はラベルの変更）があります。
[torchvision.transforms](https://pytorch.org/vision/stable/transforms.html)モジュールはよく使われる変換がいくつか用意されています。

FashionMNISTの特徴はPIL Image形式で、ラベルは整数です。
トレーニングには、特徴は正規化されたテンソル、ラベルはワンショットエンコードされたテンソルが必要です。
これらの変換を行うために、`ToTensor`と`Lambda`を使用します。


In [2]:
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

26422272it [04:16, 103081.48it/s]                              


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw


29696it [00:00, 116677.18it/s]                          


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw


4422656it [00:05, 746180.14it/s]                             


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw


6144it [00:00, 13074481.88it/s]         

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw
Processing...
Done!



/Users/shogo/miniforge3/envs/pytorch/lib/python3.8/site-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /tmp/pip-req-build-4gogm3_o/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## ToTensor()

[ToTensor](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.ToTensor)はPILイメージまたはNumPy `ndarray` を `FloatTensor` に変換し、イメージのピクセル強度値を範囲[0., 1.]でスケーリングします。

## Lambda transforms

ラムダ変換とは、ユーザーが定義した任意のラムダ関数を適用するものです。ここでは関数を定義し、整数をワンショット符号化されたテンソルに変換します。まず、サイズ10（データセットのラベル数）のゼロテンソルを作成し、[scatter](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.scatter_) を呼び出します。
これは、ラベル `y` で与えられるインデックスに `value=1` を割り当てるものです。

In [3]:
target_transform = Lambda(lambda y: torch.zeros(
    10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))

# 知識の確認

1. PyTorchのトランスフォームは以下の目的で設計されています。

* 学習に適したデータにするために、何らかの操作を行う
  > これは正しいです。

* 入力テンソルを、予測値を含む出力テンソルに変換する
  > 間違っています。変換では、トレーニングに適したデータにするために、データに何らかの操作を施します。

* データアイテムを視覚的な表現に変換する
  > 間違っています。変換では、トレーニングに適したデータにするために、データに何らかの操作を施します。

